In [1]:
import os
os.chdir('..')

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
from three_tank_data.dataset import ThreeTankDataSet
from torch.utils.data import DataLoader

ImportError: cannot import name 'ThreeTankDataSet' from 'three_tank_data.dataset' (/home/henrik/projects/sindy-autoencoder-cps/three_tank_data/dataset.py)

In [ ]:
dataset = ThreeTankDataSet(debug=True)

In [ ]:
batch_size = 1000
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=24)

In [ ]:
from sindy_autoencoder_cps.lightning_module import SINDyAutoencoder
import yaml
# MODEL_VERSION = 'freq_and_phase'
MODEL_VERSION = 'version_35'

hparams_path = f'./lightning_logs/{MODEL_VERSION}/hparams.yaml'
with open(hparams_path, 'r') as stream:
        hparam_dct = yaml.safe_load(stream)
ckpt_file_name = os.listdir(f'./lightning_logs/{MODEL_VERSION}/checkpoints/')[-1]
ckpt_file_path = f'./lightning_logs/{MODEL_VERSION}/checkpoints/{ckpt_file_name}'
model = SINDyAutoencoder.load_from_checkpoint(ckpt_file_path)
model

In [ ]:
model.SINDyLibrary.device

In [ ]:
batches = iter(dataloader)

In [ ]:
x, xdot, idxs = batches.next()
print(f'shape x: {x.shape},   shape xdot: {xdot.shape}')

In [ ]:
x_hat, xdot_hat, z, zdot, zdot_hat = model.cuda()(x.cuda(), xdot.cuda())

In [ ]:
#select random sample and compare original and recostruction
sample = 0
# sample = 477
x_sample = x[sample, :].cpu().detach().numpy()
xhat_sample = x_hat[sample, :].cpu().detach().numpy()
z_sample = z[sample, :].cpu().detach().numpy()
xdot_sample = xdot[sample, :].cpu().detach().numpy()
xdot_hat_sample = xdot_hat[sample, :].cpu().detach().numpy()
zdot_sample = zdot[sample, :].cpu().detach().numpy()
zdot_hat_sample = zdot_hat[sample, :].cpu().detach().numpy()
idx_sample = idxs[sample]

In [ ]:
import matplotlib.pyplot as plt
import constants as const
f, axarr = plt.subplots(1,2)
axarr[0].imshow(x_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))
axarr[1].imshow(xhat_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))
f, axarr = plt.subplots(1,2)
axarr[0].imshow(xdot_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))
axarr[1].imshow(xdot_hat_sample.reshape(const.PICTURE_SIZE, const.PICTURE_SIZE))

In [ ]:
zdot_sample

In [ ]:
zdot_hat_sample

In [ ]:
XI = model.XI.cpu().detach().numpy()
XI.max()

In [ ]:
import pandas as pd

In [ ]:
df_XI = pd.DataFrame(XI, columns=['z0_dot_hat', 'z1_dot_hat', 'z2_dot_hat'])
df_XI.index=model.SINDyLibrary.feature_names

In [ ]:
import plotly.express as px
print(model.SINDyLibrary.feature_names)
px.imshow(df_XI.abs())

In [ ]:
model.XI_coefficient_mask

In [ ]:
XI_mask = model.XI_coefficient_mask.cpu().detach().numpy()
df_XI_mask = pd.DataFrame(XI_mask, columns=['z0_dot_hat', 'z1_dot_hat', 'z2_dot_hat'])
df_XI_mask.index=model.SINDyLibrary.feature_names
px.imshow(df_XI_mask)

In [ ]:
dataset

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

fig = make_subplots(rows=2, cols=3, shared_xaxes=True)

for i in range(3):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=z[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'z{i}',
                             marker_color='blue', opacity=.5), row=1, col=1+i,
                    )
for i in range(3):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=zdot_hat[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot_hat{i}',
                            marker_color='blue', opacity=.4), row=2, col=1+i)
for i in range(3):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=zdot[:, i].cpu().detach().numpy(),
                      mode='markers', name=f'zdot{i}',
                             marker_color='green', opacity=.5), row=2, col=1+i)



fig.update_layout(height=600, width=1200, title_text="Side By Side Subplots")
fig.show()

In [ ]:
z_real = dataset.df.iloc[idxs][const.Z_COL_NAMES]
fig = make_subplots(rows=2, cols=3, shared_xaxes=True)

for i in range(3):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                       y=z_real.values[:, i],
                      mode='markers', name=f'z_real{i}',
                             marker_color='blue', opacity=.5), row=1, col=1+i)
for j,i in enumerate([0, 2, 1]):
    fig.add_trace(go.Scatter(x=np.array(range(z.shape[0])),
                   y=z[:, i].cpu().detach().numpy(),
                  mode='markers', name=f'z{i}',
                         marker_color='green', opacity=.5), row=2, col=1+j,
                )


fig.update_layout(height=600, width=1200, title_text="Side By Side Subplots")
fig.show()

In [ ]:
[(val, i) for val,i in enumerate(['a', 'b', 'c'])]

In [ ]:
z_real = dataset.df.iloc[idxs][const.Z_COL_NAMES]
fig = make_subplots(rows=2, cols=3)

for real, pred in zip([0, 1, 2], [0, 2, 1]):
    fig.add_trace(go.Scatter(x=z_real.values[:, real],
                       y=z[:, pred].cpu().detach().numpy(),
                      mode='markers', name=f'z_real{i}',
                             marker_color='blue', opacity=.5), row=1, col=1+real)


fig.update_layout(height=600, width=1200, title_text="Side By Side Subplots")
fig.show()

In [ ]:
model.XI_coefficient_mask